In [1]:
import pandas as pd

from aggregation import aggeregate_stance, aggregate_sentiment

In [2]:
data_after_toloka = pd.read_csv('data_after_toloka_example.tsv', sep='\t')
workers_and_skills = pd.read_csv('skils_from_toloka_example.csv')

In [3]:
data_after_toloka.head()

,INPUT:topic,INPUT:context,OUTPUT:stance,OUTPUT:sentiment,ASSIGNMENT:worker_id
0,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,neutral,neutral,c6465efd1f64fe3665a1b93b2bb3c30e
1,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,neutral,neutral,f65a8535bf796dfae5a3f127a8ce508b
2,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,favor,neutral,d8fd4ae57fcdbe20cf2c62eac37de6ac
3,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,favor,negative,2499c961121ce87cd26a7611d2c604f7
4,ЛГБТК+,транса в текстовом виде ура схпхвх,neutral,neutral,2f3abb7925f1ebc67aeba79bd827ef08


In [4]:
workers_and_skills.head()

,worker_id,skill
0,6117d4372bf412a105b22c71ba074df9,40
1,ae03f39c26dc7e36252c5f33cb693b79,60
2,442c8cc730a03a5645eb527b91848f74,50
3,849b20e3852624902920b97c9116a2c9,75
4,c780ec01f0bcc577484bff1c79b37878,50


In [5]:
data_after_toloka.rename(columns={'INPUT:topic':'topic', 'INPUT:context':'content', 'OUTPUT:stance':'stance', 'OUTPUT:sentiment':'sentiment', 'ASSIGNMENT:worker_id':'worker_id'}, inplace=True)

In [6]:
data_after_toloka.head()

,topic,content,stance,sentiment,worker_id
0,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,neutral,neutral,c6465efd1f64fe3665a1b93b2bb3c30e
1,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,neutral,neutral,f65a8535bf796dfae5a3f127a8ce508b
2,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,favor,neutral,d8fd4ae57fcdbe20cf2c62eac37de6ac
3,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,favor,negative,2499c961121ce87cd26a7611d2c604f7
4,ЛГБТК+,транса в текстовом виде ура схпхвх,neutral,neutral,2f3abb7925f1ebc67aeba79bd827ef08


In [7]:
data_after_toloka.shape

(60, 5)

In [8]:
len(data_after_toloka.content.unique())

20

In [9]:
def to_int(s):
    return int(s)

In [10]:
data_after_toloka = pd.merge(data_after_toloka, workers_and_skills, on="worker_id", how='left')
data_after_toloka.fillna(50, inplace=True)
data_after_toloka['weight'] = data_after_toloka.skill.apply(to_int) / 100

data_after_toloka.head()

,topic,content,stance,sentiment,worker_id,skill,weight
0,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,neutral,neutral,c6465efd1f64fe3665a1b93b2bb3c30e,40.0,0.40
1,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,neutral,neutral,f65a8535bf796dfae5a3f127a8ce508b,50.0,0.50
2,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,favor,neutral,d8fd4ae57fcdbe20cf2c62eac37de6ac,65.0,0.65
3,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,favor,negative,2499c961121ce87cd26a7611d2c604f7,55.0,0.55
4,ЛГБТК+,транса в текстовом виде ура схпхвх,neutral,neutral,2f3abb7925f1ebc67aeba79bd827ef08,45.0,0.45


In [11]:
data_stance = data_after_toloka[['topic', 'content', 'stance', 'worker_id', 'skill', 'weight']]
data_sentiment = data_after_toloka[['topic', 'content', 'sentiment', 'worker_id', 'skill', 'weight']]

In [12]:
data_stance_aggregated = aggeregate_stance(data_stance)
data_sentiment_aggregated = aggregate_sentiment(data_sentiment)

In [13]:
data_stance_aggregated.head()

,topic,content,stance
2,ЛГБТК+,"модерн ау, в которой Миша и Сережа -- ЛГБ...",error
5,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,favor
6,ЛГБТК+,- Людмила Петрановская о гомофобии в России: ...,error
9,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,favor
12,ЛГБТК+,Как ЦК КПСС приглашал в Москву берлинских гей...,neutral


In [14]:
data_sentiment_aggregated.head()

,topic,content,sentiment
0,ЛГБТК+,"модерн ау, в которой Миша и Сережа -- ЛГБ...",neutral
3,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,neutral
7,ЛГБТК+,- Людмила Петрановская о гомофобии в России: ...,neutral
9,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,neutral
12,ЛГБТК+,Как ЦК КПСС приглашал в Москву берлинских гей...,neutral


In [15]:
aggregated_data = pd.merge(data_stance_aggregated, data_sentiment_aggregated, on=['topic', 'content'])

aggregated_data.head()

,topic,content,stance,sentiment
0,ЛГБТК+,"модерн ау, в которой Миша и Сережа -- ЛГБ...",error,neutral
1,ЛГБТК+,шрамы: родимые пятна: плохое зрение: поц...,favor,neutral
2,ЛГБТК+,- Людмила Петрановская о гомофобии в России: ...,error,neutral
3,феминизм,-- Джен/Дженни -- для близких Зи!! -- изучаю ...,favor,neutral
4,ЛГБТК+,Как ЦК КПСС приглашал в Москву берлинских гей...,neutral,neutral
